In [ ]:
from docksuitex import Protein, Ligand, PocketFinder, VinaDocking, AD4Docking
from docksuitex.utils import fetch_pdb, fetch_sdf, parse_vina_log_to_csv, parse_ad4_dlg_to_csv, clean_temp_folder
clean_temp_folder()

In [ ]:
# Fetch and prepare protein
Protein_path = fetch_pdb(pdbid = "1HVR")
prot = Protein(Protein_path)
prot.prepare(fix_pdb =True, remove_heterogens=True, add_hydrogens=True, remove_water=True, add_charges=True, preserve_charge_types=None) #Default parameters
# prot.save_pdbqt(save_to="1HVR.pdbqt")
prot.view_molecule()

In [ ]:
# Find pockets using p2rank
pf = PocketFinder(prot)
pockets = pf.run()
# pf.save_report(save_to="p2rank_report")
pockets

In [ ]:
# Fetch and prepare ligand
Ligand_path = fetch_sdf(cid = "338")
lig = Ligand(Ligand_path)
lig.prepare(minimize=None,  remove_water=True, add_hydrogens=True, add_charges=True, preserve_charge_types=None) #Default parameters
# lig.save_pdbqt(save_to="338.pdbqt")
lig.view_molecule()

## Perform protein ligand docking at the first pocket

#### Using AutoDock Vina


In [ ]:
center = pockets[0]['center']

vina = VinaDocking(
    receptor=prot,
    ligand=lig,
    grid_center=center,
    
    # Default Parameters
    grid_size=(20,20,20),
    exhaustiveness=8,
    num_modes=9
)
vina.run()
vina.view_results()
vina.save_results(save_to="vina_results")

# Parse Vina results
parse_vina_log_to_csv(results_dir="vina_results", output_csv="vina_results/vina_summary.csv")


#### Using AutoDock4

In [ ]:
# Perform docking with the first pocket
# Using AutoDock4
ad4 = AD4Docking(
    receptor=prot,
    ligand=lig,
    grid_center=pockets[0]['center'],

    #Default Parameters
    grid_size=(60,60,60),
    spacing=0.375,     
    dielectric=-0.1465, 
    smooth=0.5,          
    ga_pop_size=150,
    ga_num_evals=2500000,
    ga_num_generations=27000,
    ga_elitism=1,
    ga_mutation_rate=0.02,
    ga_crossover_rate=0.8,
    ga_run=10,
    rmstol=2.0,
)

ad4.run()
ad4.view_results()
ad4.save_results(save_to="ad4_results")

# Parse AD4 results
parse_ad4_dlg_to_csv(results_dir="ad4_results", output_csv="ad4_results/ad4_summary.csv")